# Recent Images Microservice

This microservice can be used to request multiple tile urls, thumbnail urls and metadata from Google Earth Engine (GEE) in a fast, asynchronus way.

### Overview

On the GFW map page users can currently use the *Sentinel service* to inspect a single satellite image tile for a given area of interest when at high zoom level. The tile url of the image with the lowest ```cloud_score``` within a selected date range is requested from GEE and used on the front end to display the image.

This microservice build on this idea by requesting all images in the date range asynchronously as well as teh corresponding thumbnail urls and associated metadata.

***

### Intended Useage

The microservice is intented to be implemented in 3 stages on the front end:

1) First, do a GET request for the metadata from a GEE image collection (ordered by CLOUDY_PIXEL_PERCENTAGE), as well as the tile url for the best image (i.e. the one with the lowest CLOUDY_PIXEL_PERCENTAGE score)

2) Next POST a payload containing the ```'source'``` values of all images to the ```/v1/recent-tiles/tiles``` endpoint, which returns the tile urls for the associated images.

3) Finally POST the same payload to the ```/v1/recent-tiles/thumbs``` endpoint, which returns all associated thumbnail urls.

***


### Endpoints

The microservice has 3 endpoints:

**/v1/recent-tiles** (GET)
- *Returns tile metadata and a single tile url*
- url params: ```lat, lon, start, end```

**/v1/recent-tiles/tiles** (POST)
- *Returns all tile urls*
- payload: ```list of JSON objects containing 'source' value```

**/v1/recent-tiles/thumbs** (POST)
- *Returns all thumb urls*
- payload: ```list of JSON objects containing 'source' value```





In [1]:
import folium
import os
import ee
import json
import requests
import math
from scipy import misc
import shutil
from pprint import pprint
import requests
from IPython.display import Image


# Request 1

First query requests data for the following params, for example:

```json
    url = f"https://staging-api.globalforestwatch.org/v1/recent-tiles"
    params = {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2016-01-08"}
```

And returns an object of the following format:

```json
"data": [
    {
        "attributes": {
        "boundary_tiles": "https://earthengine.googleapis.com/map/4b1b...",
        "cloud_score": 13.1214,
        "date_time": "2017-01-31 11:52:11Z",
        "instrument": "2017-01-31 11:52:11Z",
        "source": "COPERNICUS/S2/20170131T115211_20170131T115306_T28RCS",
        "thumbnail_url": Null,
        "tile_url": Null
        },
        "id": Null,
        "type": "recent_tiles_url"
    }

```

There may be multiple values in the ```'data'``` list (expect one image per 8 days approx.)


In [10]:
%%time
# Request metadata (6 month period)

url = f"https://staging-api.globalforestwatch.org/v1/recent-tiles"
params= {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2016-07-01"}
r = requests.get(url, params=params)
r.status_code

data = r.json().get('data')

pprint(data[0:3])

[{'attributes': {'boundary_url': 'https://earthengine.googleapis.com/map/4b1b9c6f82d50796562521502bc4d9a2/{z}/{x}/{y}?token=95a9ff889c620d95c60f70d67ea05903',
                 'cloud_score': 0.4339,
                 'date_time': '2016-03-17 11:53:14Z',
                 'instrument': 'Sentinel-2A',
                 'source': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS',
                 'thumbnail_url': None,
                 'tile_url': 'https://earthengine.googleapis.com/map/5d9dac31193dfb4ef1003021827755cc/{z}/{x}/{y}?token=78d9db699b548c9130a72a3f49b28e49'},
  'id': None,
  'type': 'recent_tiles_data'},
 {'attributes': {'boundary_url': 'https://earthengine.googleapis.com/map/4b1b9c6f82d50796562521502bc4d9a2/{z}/{x}/{y}?token=95a9ff889c620d95c60f70d67ea05903',
                 'cloud_score': 3.1135,
                 'date_time': '2016-06-05 11:53:14Z',
                 'instrument': 'Sentinel-2A',
                 'source': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28

# Creating Payload for POST

On the front end, the response must be used to buld a list of objects containing the ```'source'``` values of the images in the collection.

*e.g.*

```json
{'source_data':
    
    [
        {"source": "COPERNICUS/S2/20170131T11..."},
        {"source": "COPERNICUS/S2/20170131T12..."},
        {"source": "COPERNICUS/S2/20170131T13..."},    
        
        ...
      
        {"source": "COPERNICUS/S2/20170131T1N..."}
    ]
 }

```

### IMPORTANT

- *The json must have the key ```'source_data'```!*

- Must have a the following header: ```headers={'Content-Type': 'application/json'}```

In [3]:
#example of how to construct the source list

source_list = []

for d in data:
    source_list.append({'source': d.get('attributes').get('source')}) 
source_list[:]

[{'source': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS'},
 {'source': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28RCS'},
 {'source': 'COPERNICUS/S2/20160416T115220_20160416T172059_T28RCS'},
 {'source': 'COPERNICUS/S2/20160506T115225_20160506T184707_T28RCS'},
 {'source': 'COPERNICUS/S2/20160327T115315_20160327T172523_T28RCS'},
 {'source': 'COPERNICUS/S2/20160615T115223_20160615T183608_T28RCS'},
 {'source': 'COPERNICUS/S2/20160117T115842_20160117T172446_T28RCS'},
 {'source': 'COPERNICUS/S2/20160526T115225_20160526T184022_T28RCS'},
 {'source': 'COPERNICUS/S2/20160406T115217_20160406T185930_T28RCS'},
 {'source': 'COPERNICUS/S2/20160625T115243_20160625T200443_T28RCS'},
 {'source': 'COPERNICUS/S2/20160426T115223_20160426T184333_T28RCS'},
 {'source': 'COPERNICUS/S2/20160107T115414_20160108T134003_T28RCS'},
 {'source': 'COPERNICUS/S2/20160516T115226_20160516T185815_T28RCS'},
 {'source': 'COPERNICUS/S2/20160216T115313_20160216T173530_T28RCS'},
 {'source': 'COPERNICUS/S2/2016030

In [17]:
%%time
# Request tiles using the payload

url = f"https://staging-api.globalforestwatch.org/v1/recent-tiles/tiles"
payload = {'source_data':source_list}
r = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
r.status_code

tile_data = r.json().get('data').get('attributes')

pprint(tile_data[0:3])

[{'source_id': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS',
  'tile_url': 'https://earthengine.googleapis.com/map/5d9dac31193dfb4ef1003021827755cc/{z}/{x}/{y}?token=ac504f10bef46a8d61b40ddeef62110e'},
 {'source_id': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28RCS',
  'tile_url': 'https://earthengine.googleapis.com/map/73f2aa15a7e70eb5a99f3df87d06a979/{z}/{x}/{y}?token=dce55865ba9c2d9ea3daf0e359bd9c1c'},
 {'source_id': 'COPERNICUS/S2/20160416T115220_20160416T172059_T28RCS',
  'tile_url': 'https://earthengine.googleapis.com/map/0dfc07c304574e3ac4b2d8a3c37e4374/{z}/{x}/{y}?token=58448e131ae67b98fa9fff4481addf0d'}]
CPU times: user 21 ms, sys: 3.72 ms, total: 24.8 ms
Wall time: 1.62 s


### Tiles Response

At this stage the data now looks like:

```json
{'data': {
    'attributes': [{"source": "COPERNICUS/S2/20170131T11...",
                    "tile_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },{
                    "source": "COPERNICUS/S2/20170131T12...",
                    "tiles_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },           
                      ...
      
                    {
                    "source": "COPERNICUS/S2/20170131T1N...",
                    "tiles_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   }],
          'id': None,
          'type': 'recent_tiles_url'}}
          }
}
```

In [15]:
%%time

# Request thumbs using the payload

url = f"https://staging-api.globalforestwatch.org/v1/recent-tiles/thumbs"
payload = {'source_data':source_list}
r = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
r.status_code

thumb_data = r.json().get('data').get('attributes')

pprint(thumb_data[0:3])

[{'source': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS',
  'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=f8d1d5c66a6d3e7dc30002e663242830&token=337de0abb7ac3d4c8d8a545e1c72311b'},
 {'source': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28RCS',
  'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=fbcea1cc967a46d14c48c2367c0bf5a3&token=83355743d9feec7f72205f9104420b0f'},
 {'source': 'COPERNICUS/S2/20160416T115220_20160416T172059_T28RCS',
  'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=1bfbf427df20fd31c68ab9f575ff04f2&token=1ce069e54de40d89f4b5053d14b0c337'}]
CPU times: user 23.3 ms, sys: 4.64 ms, total: 27.9 ms
Wall time: 2.81 s


### Thumbs Response

At this stage the data now looks like:

```json
{'data': {
    'attributes': [{"source": "COPERNICUS/S2/20170131T11...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },{
                    "source": "COPERNICUS/S2/20170131T12...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },           
                      ...
      
                    {
                    "source": "COPERNICUS/S2/20170131T1N...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   }],
          'id': None,
          'type': 'recent_thumbs_url'}}
          }
}
```

# Image Example for Map

In [6]:
# Returned First Image and Boundary

dt = data[0].get('attributes').get('date_time')
boundary = data[0].get('attributes').get('boundary_url')
sentinel_image = data[0].get('attributes').get('tile_url')

In [7]:
sentinel_map = folium.Map(location=[float(params['lon']), float(params['lat'])], zoom_start=9, tiles='Mapbox Bright' )
sentinel_map.add_tile_layer(tiles=sentinel_image, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map.add_tile_layer(tiles=boundary, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map

In [8]:
# Corresponding Thumbnail

thumbnail_image = thumb_data.get('attributes')[0].get('thumbnail_url')
Image(url=thumbnail_image)